In [ ]:
import numpy as np
import pandas as pd
from scipy import io
import os


将.mat文件转换成.npz
并把时间序列reshape成长度为3000（也就是30s）的大小
并把标签转换为one-hot编码

In [ ]:
path1 = 'F:\EDFProcessCode\st_edfmat'
path2 = 'F:\EDFProcessCode\st_edf_labels'

for file_name1,file_name2 in zip(os.listdir(path1),os.listdir(path2)):
    path11 = 'H:\\EEG\\st_edfmat\\'+file_name1
    path22 = 'F:\\EDFProcessCode\\st_edf_labels\\'+file_name2
    
    t = pd.read_csv(path22,header=None)
    df = pd.DataFrame(t)
    df = np.array(df)
    df_b = df.tolist()
    for i in range(len(df_b)):
        if df_b[i][0] == 6:
            df_b[i][0]=0
        elif df_b[i][0] == 5:
            df_b[i][0]=4
        elif df_b[i][0] == 4:
            df_b[i][0]=1
        elif df_b[i][0] == 3:
            df_b[i][0]=2
        elif df_b[i][0] == 2:
            df_b[i][0]=3
        elif df_b[i][0] == 1:
            df_b[i][0]=3
    df_b = pd.DataFrame(df_b,columns=[3000])
    

    data = io.loadmat(path11)
    data = data['record'][1]
    data = data.reshape((-1,3000))
    data = pd.DataFrame(data)

    dd_b = pd.concat([data,df_b],axis=1)
    dd_b = dd_b.drop(dd_b[dd_b[3000] == -1].index)


    x = dd_b.iloc[:,:-1]
    bdd = dd_b.iloc[:,-1:]
    bdd = np.array(bdd).tolist()
    for l in range(len(bdd)):
        if bdd[l][0] == 0:
            bdd[l]=[1,0,0,0,0]
        elif bdd[l][0] == 1:
            bdd[l]=[0,1,0,0,0]
        elif bdd[l][0] == 2:
            bdd[l]=[0,0,1,0,0]
        elif bdd[l][0] == 3:
            bdd[l]=[0,0,0,1,0]
        elif bdd[l][0] == 4:
            bdd[l]=[0,0,0,0,1]

    y = pd.DataFrame(bdd)
    
    np.savez('H:\\EEG\\truehot\\'+file_name1[:6]+'.npz', x=x,y=y)

将所有受试者的.npz文件合并

In [ ]:

data_all1 = [np.load('H:\\EEG\\SC4001.npz')]
merged_data = {}
for data in data_all1:
    for k, v in data.items():
        merged_data.update({k: v}) 
        
patha = 'H:\\EEG\\testhot'
file_list = []
for file_name in os.listdir(patha):
    file_list.append(file_name)        

data_all = [np.load('H:\\EEG\\testhot\\'+fname) for fname in file_list]
for data in data_all:
    for k, v in data.items():
        merged_data[k] = np.r_[merged_data[k],v]
np.savez('H:\\EEG\\test3file.npz', **merged_data)